# Ejercicio 8: Bases de Datos Vectoriales

Las bases de datos vectoriales permiten almacenar y recuperar información representada como vectores en espacios de alta dimensión. Primero vamos a revisar los fundamentos matemáticos en los que se basan.

## 1. Espacios Vectoriales

Cada documento, imagen, o consulta se representa como un vector real en un espacio ℝ^n:

$\[ \vec{d} = [d_1, d_2, \dots, d_n] \in \mathbb{R}^n \]$

Donde $\( n \)$ suele ser 384, 768 o 1536, dependiendo del modelo de embeddings utilizado.

In [2]:
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [nltk]1/2 [nltk]

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim.downloader as api
import numpy as np
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/paullora/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/paullora/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data
newsgroupsdocs_df = pd.DataFrame(newsgroupsdocs, columns=['raw'])
newsgroupsdocs_df

,raw
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess_doc(doc):
    words = word_tokenize(doc.lower())
    words_filtered = [word for word in words if word not in stop_words]
    return ' '.join(words_filtered)

In [31]:
newsgroupsdocs_df['processed'] = newsgroupsdocs_df['raw'].apply(preprocess_doc)

In [32]:
newsgroupsdocs_df

,raw,processed
0,\n\nI am sure some bashers of Pens fans are pr...,sure bashers pens fans pretty confused lack ki...
1,My brother is in the market for a high-perform...,brother market high-performance video card sup...
2,\n\n\n\n\tFinally you said what you dream abou...,finally said dream . mediterranean ? ? ? ? new...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think ! 's scsi card dma transfers disks ... s...
4,1) I have an old Jasmine drive which I cann...,1 ) old jasmine drive use new system . underst...
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,dn > : nyeda @ cnsvax.uwec.edu ( david nye ) d...
18842,\nNot in isolated ground recepticles (usually ...,isolated ground recepticles ( usually unusual ...
18843,I just installed a DX2-66 CPU in a clone mothe...,"installed dx2-66 cpu clone motherboard , tried..."
18844,\nWouldn't this require a hyper-sphere. In 3-...,"would n't require hyper-sphere . 3-space , 4 p..."


In [35]:
model = api.load("word2vec-google-news-300")

In [66]:
def embedding_doc(doc):
    words = word_tokenize(doc)
    doc_embeddings = [model[word] for word in words if word in model]
    return np.mean(doc_embeddings, axis=0)

In [67]:
newsgroupsdocs_df['embeddings'] = newsgroupsdocs_df['processed'].apply(embedding_doc)

/Users/paullora/Desktop/EPN/ir/ir-2025A/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paullora/Desktop/EPN/ir/ir-2025A/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [68]:
newsgroupsdocs_df

,raw,processed,embeddings
0,\n\nI am sure some bashers of Pens fans are pr...,sure bashers pens fans pretty confused lack ki...,"[0.061880082, 0.07067529, -0.02666849, 0.08532..."
1,My brother is in the market for a high-perform...,brother market high-performance video card sup...,"[-0.021407517, 0.010921337, 0.018396448, 0.005..."
2,\n\n\n\n\tFinally you said what you dream abou...,finally said dream . mediterranean ? ? ? ? new...,"[0.022130935, 0.016848527, 0.03843176, 0.07588..."
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think ! 's scsi card dma transfers disks ... s...,"[0.030354667, -0.006498449, 0.0025616814, 0.04..."
4,1) I have an old Jasmine drive which I cann...,1 ) old jasmine drive use new system . underst...,"[0.062327992, -0.0061734286, 0.033541594, 0.07..."
...,...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,dn > : nyeda @ cnsvax.uwec.edu ( david nye ) d...,"[-0.073149584, 0.061401464, -0.00469598, 0.114..."
18842,\nNot in isolated ground recepticles (usually ...,isolated ground recepticles ( usually unusual ...,"[0.030321907, -0.0007019043, -0.044720817, 0.0..."
18843,I just installed a DX2-66 CPU in a clone mothe...,"installed dx2-66 cpu clone motherboard , tried...","[0.0029458804, 0.047783054, -0.03366774, 0.145..."
18844,\nWouldn't this require a hyper-sphere. In 3-...,"would n't require hyper-sphere . 3-space , 4 p...","[-0.022285461, -0.028199514, 0.035079956, 0.09..."


In [69]:
query = "computer graphics"
query_embedding = embedding_doc(query)
query_embedding

array([ 0.171875  , -0.17041016,  0.10131836,  0.18603516, -0.08813477,
        0.16772461,  0.03759766,  0.00927734,  0.08782959,  0.05236816,
       -0.02734375,  0.01287842, -0.14038086, -0.09143066,  0.0291748 ,
       -0.05541992, -0.05102539,  0.09829712,  0.11169434, -0.29638672,
        0.00634766,  0.14355469,  0.08668137,  0.04241943,  0.09042358,
       -0.03800201,  0.06970215,  0.08349609,  0.0256958 , -0.2211914 ,
       -0.21484375,  0.03857422, -0.171875  ,  0.12359619, -0.17016602,
       -0.109375  , -0.14453125,  0.26367188,  0.00921631,  0.03381348,
        0.03417969,  0.16503906,  0.15722656,  0.22363281,  0.05773926,
       -0.03808594,  0.01489258,  0.0637207 ,  0.06494141,  0.11437607,
       -0.2553711 , -0.12353516,  0.05297852, -0.18310547, -0.21875   ,
        0.24072266,  0.1564331 , -0.12210083,  0.01843262,  0.14160156,
        0.05957031,  0.16577148, -0.2536621 ,  0.07653809, -0.16796875,
        0.11523438, -0.11499023,  0.25048828, -0.18164062,  0.03

## 2. Medidas de Similitud

El principio básico de una base vectorial es buscar elementos cuyo vector esté "cerca" del vector de consulta. Existen varias formas de medir esta cercanía:

### a. Distancia Euclidiana (L2)

$\[ \text{dist}(⇡\vec{q}, \vec{d}) = \sqrt{\sum_{i=1}^n (q_i - d_i)^2} \]$

Utilizada cuando los vectores no están normalizados. Implementada por defecto en `FAISS` con `IndexFlatL2`.

### b. Similitud Coseno

$\[ \cos(\theta) = \frac{\vec{q} \cdot \vec{d}}{\|\vec{q}\| \cdot \|\vec{d}\|} \]$

Esta métrica es ideal cuando se desea medir ángulos (dirección) en lugar de magnitudes. Se usa en `ChromaDB` y también puede simularse en FAISS si los vectores están normalizados.

Existe una relación entre ambas (cuando los vectores están normalizados):
$\[ \text{dist}_{\text{L2}}^2 = 2 - 2 \cdot \cos(\theta) \]$

In [85]:
from sklearn.metrics.pairwise import cosine_similarity

embeddings_matrix = newsgroupsdocs_df['embeddings'].to_numpy().reshape(-1, 1)  # Ensure the shape is (n_docs, 300)

# embeddings_matrix.shape
query_embedding_2d = query_embedding.reshape(-1, 1)  # shape: (1, 300)

embeddings_matrix.shape

# Use the correct 2D numpy array for embeddings
results = cosine_similarity(embeddings_matrix, query_embedding_2d)
# results = results.flatten()  # shape: (n_docs,)

# results

ValueError: setting an array element with a sequence.

In [3]:
from numpy.linalg import norm

dist1 = norm(query - doc1)
dist2 = norm(query - doc2)

print("Distancia Euclidiana a doc1:", dist1)
print("Distancia Euclidiana a doc2:", dist2)

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

sim1 = cosine_similarity(query, doc1)
sim2 = cosine_similarity(query, doc2)

print("Similitud coseno con doc1:", sim1)
print("Similitud coseno con doc2:", sim2)

Distancia Euclidiana a doc1: 0.2449489742783178
Distancia Euclidiana a doc2: 0.24494897427831785
Similitud coseno con doc1: 0.8951435925492909
Similitud coseno con doc2: 0.8846153846153845


## 3. Normalización de Vectores

Muchos sistemas normalizan los vectores para que su norma sea 1:

$\[ \hat{\vec{v}} = \frac{\vec{v}}{\|\vec{v}\|} \]$

Esto transforma la distancia Euclidiana en una función directa de la similitud coseno, facilitando búsquedas eficientes y comparables.

In [4]:
def normalize(v):
    return v / norm(v)

q_norm = normalize(query)
d1_norm = normalize(doc1)
d2_norm = normalize(doc2)

print("Vector normalizado q:", q_norm)
print("Similitud coseno post-normalización (dot):", np.dot(q_norm, d1_norm), np.dot(q_norm, d2_norm))

# Relación teórica: dist² = 2 - 2cos(θ)
dot = np.dot(q_norm, d1_norm)
euclidean_sq = norm(q_norm - d1_norm)**2
print("2 - 2cos(theta):", 2 - 2 * dot)
print("Distancia euclidiana al cuadrado:", euclidean_sq)

Vector normalizado q: [0.19611614 0.58834841 0.78446454]
Similitud coseno post-normalización (dot): 0.895143592549291 0.8846153846153845
2 - 2cos(theta): 0.20971281490141802
Distancia euclidiana al cuadrado: 0.20971281490141774


## 4. Indexación y Aceleración

Buscar en millones de vectores directamente es costoso $(\( O(n \cdot d) \))$. Se usan estructuras aproximadas para acelerar:

### a. IVF (Inverted File Index)
- Aplica clustering (K-means) a los vectores.
- Durante la búsqueda, se consulta solo un subconjunto de clústeres.

### b. HNSW (Hierarchical Navigable Small World)
- Construye un grafo jerárquico de vecinos más cercanos.
- Permite búsquedas logarítmicas eficientes.